In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml("mnist_784", as_frame = False)
print(type(mnist))

<class 'sklearn.utils._bunch.Bunch'>
